In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Определить потенциально отточных клиентов

    У нашей компании есть клиенты.
    Они оплачивают наши услуги по различным тарифам. 
    У этих тарифов разная стоимость, разный срок действия и разные условия. Подробнее можно ознакомится здесь - https://www.b2b-center.ru/app/tariffs/?group=supplier.
    Нашей компании важно знать, собирается ли клиент продлевать свой тариф или же перестанет пользоваться нашими услугами ("оттечет"). 
    Большинство тарифов квартальные и поэтому далее будем говорить об оплатах в рамках квартала.
    
    
    

# Тестовое задание
    Тестовое задание моделирует (упрощенно) реальную задачу прогноза оплаты/оттока клиента и формат задачи выполнен аналогично задаче на kaggle.
    Допустим мы находимся в 1 декабре 2022 года (все тренировочные данные заканчиваются этой датой)
    
    Нам важно знать кто из клиентов заплатит нам в 1 квартале 2023 года. Точное значение получить невозможно, поэтому хочется оценить "вероятность" этой оплаты.
    
    
    В качестве проверочной метрики будет использоваться roc_auc_score, но интересно будет обсудить и альтернативные подходы к выбору метрики.

# Данные

In [5]:

""" 
В файле train_action_df.csv содержится информация о важнейших действиях на нашем сайте за последние 2 года. 

поля:
    user_id - уникальный идентификатор клиента
    action_type - 5 типов важнейших действий на нашем сайте. Категориальная переменная (action_4 НЕ обязательно важнее, чем action_3 и т.д.)
    action_date - дата совершения действия
    cnt_actions - количество действий данного типа, совершенных в эту дату

"""

train_action_df = pd.read_csv('train_action_df.csv')
train_action_df.head()

,user_id,action_type,action_date,cnt_actions
0,user_26091,action_2,2020-12-04,1
1,user_26091,action_2,2020-12-17,1
2,user_26091,action_2,2020-12-21,1
3,user_26091,action_2,2021-01-13,1
4,user_26091,action_2,2021-01-15,1


In [6]:
""" 
В файле train_pay_df содержится информация об оплатах за последние 2 года. 

поля:
    user_id - уникальный идентификатор клиента
    pay_date - дата совершения оплаты (НЕ всегда соответствует оплаченному периоду!!!)
    year - год действия тарифа
    quarter - квартал действия тарифа
    tariff - обезличенный тип тарифа 
    

Небольшие пояснения.
Довольно часто клиенты оплачивают тариф заранее. Например, 1 ноября 2022 года могут оплатить 1 квартал 2023 года.
    
"""

train_pay_df = pd.read_csv('train_pay_df.csv')
train_pay_df.head()

,user_id,pay_date,year,quarter,period,tariff
0,user_50838,2021-01-03,2021,1,205,tariff_15
1,user_56500,2021-01-05,2021,1,205,tariff_20
2,user_50795,2021-01-05,2021,1,205,tariff_25
3,user_4718,2021-01-10,2021,1,205,tariff_10
4,user_22255,2021-01-10,2021,1,205,tariff_10


# Результат

In [7]:
# в файле test_df находятся клиенты, для которых необходимо сделать прогноз
test_df = pd.read_csv('test_df.csv')

# результатом модель должна выдать вероятность (proba) для каждой строчки из этого файла
# я заполню случайно для понимания формата файла
test_df['proba'] = np.random.uniform(size = test_df.shape[0])

In [8]:
test_df.head()

,user_id,year,quarter,period,proba
0,user_44331,2022,4,212,0.156357
1,user_49203,2022,4,212,0.522755
2,user_21597,2022,4,212,0.333774
3,user_36314,2022,4,212,0.666885
4,user_45092,2022,4,212,0.277892


In [9]:
# в качестве результата достаточно прислать файл с 2-мя столбцами
test_df[['user_id','proba']].to_csv('predict_df.csv', index = False)

# Связь test_df и обучающих данных

In [10]:
# test_df может быть получен из train_pay_df следующим скриптом:
copy_test_df = train_pay_df[(train_pay_df['year']==2022) & 
             (train_pay_df['quarter']==4)][['user_id','year','quarter','period']].drop_duplicates().reset_index(drop=True)

assert copy_test_df.shape[0]==copy_test_df.merge(test_df).shape[0]

In [11]:
copy_test_df.head()

,user_id,year,quarter,period
0,user_44331,2022,4,212
1,user_49203,2022,4,212
2,user_21597,2022,4,212
3,user_36314,2022,4,212
4,user_45092,2022,4,212


# Проверка

In [12]:
# при проверке я возьму реальные данные об оплатах каждым пользователем 
# хотя бы 1 раз period = 213 (year = 2023, quarter = 1)
# Если пользователь заплатил, то target = 1, если нет = 0.

# для примера сейчас тоже заполню случайно
test_df['target'] = np.random.randint(0, 2, size = test_df.shape[0])

print('Финальная метрика', roc_auc_score(test_df['target'], test_df['proba']))

Финальная метрика 0.49640501957325256


# Решение

## Кроме файла с прогнозом (predict_df.csv) крайне желательно прислать ноутбук или скрипт, которым это решение было получено для понимания уровня кандидата

# Мое решение

In [14]:
# Dataframe with features
print(train_action_df.info())
print(train_pay_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2881365 entries, 0 to 2881364
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      object
 1   action_type  object
 2   action_date  object
 3   cnt_actions  int64 
dtypes: int64(1), object(3)
memory usage: 87.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137849 entries, 0 to 137848
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   137849 non-null  object
 1   pay_date  137849 non-null  object
 2   year      137849 non-null  int64 
 3   quarter   137849 non-null  int64 
 4   period    137849 non-null  int64 
 5   tariff    137849 non-null  object
dtypes: int64(3), object(3)
memory usage: 6.3+ MB
None


In [15]:
train_action_df['action_date'] = pd.to_datetime(train_action_df['action_date']).dt.date
train_pay_df['pay_date'] = pd.to_datetime(train_pay_df['pay_date']).dt.date

In [16]:
from sklearn.preprocessing import OneHotEncoder

# Choosing OneHotEncoder because I don't know if action1 is less important than action4
OHE = OneHotEncoder(handle_unknown='ignore',sparse=False)

# Encoding tariffs
encoded_tarrifs = OHE.fit_transform(train_pay_df[['tariff']])
encoded_tarrifs_int = encoded_tarrifs.astype(int)
encoded_tarrifs_df = pd.DataFrame(encoded_tarrifs_int, columns=OHE.get_feature_names_out(['tariff']))
encoded_tarrifs_df.columns = [col.replace('tariff_tariff','tariff') for col in encoded_tarrifs_df.columns]
train_pay_df_encoded = pd.concat([train_pay_df,encoded_tarrifs_df],axis=1)



def get_start_of_quarter(row):
    period = pd.Period(f"{row['year']}Q{row['quarter']}")
    return period.to_timestamp()

# Create a new column with the start date of each quarter
train_pay_df_encoded['tariff_start'] = train_pay_df_encoded.apply(get_start_of_quarter, axis=1)

# Calculating how early user bought the tariff before the start of tariff
train_pay_df_encoded['earliness_of_pay_date'] = train_pay_df_encoded['tariff_start'] - pd.to_datetime(train_pay_df_encoded['pay_date'])
train_pay_df_encoded['earliness_of_pay_date'] = train_pay_df_encoded['earliness_of_pay_date'].dt.days

# Deleting unnecessary columns
train_pay_df_encoded = train_pay_df_encoded.drop(['tariff','period','year','quarter','tariff_start','pay_date'], axis=1)

# Encoding action types
encoded_actions = OHE.fit_transform(train_action_df[['action_type']])
encoded_actions_int = encoded_actions.astype(int)
encoded_actions_df = pd.DataFrame(encoded_actions_int, columns=OHE.get_feature_names_out(['action_type']))

# train_action_df with OneHotEncoder performed on "action_type"
train_action_df_encoded = pd.concat([train_action_df[['user_id','action_date','cnt_actions']],encoded_actions_df],axis=1) 

# Multiply the 'amount_of_actions' across each one-hot encoded column
for col in OHE.get_feature_names_out(['action_type']):
    train_action_df_encoded[col] = train_action_df_encoded[col] * train_action_df_encoded['cnt_actions']
del train_action_df_encoded['cnt_actions']
train_action_df_encoded.columns = [col if col not in OHE.get_feature_names_out(['action_type']) else 'action_'+col[-1] for col in train_action_df_encoded.columns]
train_action_df_encoded

,user_id,action_date,action_1,action_2,action_3,action_4,action_5
0,user_26091,2020-12-04,0,1,0,0,0
1,user_26091,2020-12-17,0,1,0,0,0
2,user_26091,2020-12-21,0,1,0,0,0
3,user_26091,2021-01-13,0,1,0,0,0
4,user_26091,2021-01-15,0,1,0,0,0
...,...,...,...,...,...,...,...
2881360,user_20341,2022-10-24,2,0,0,0,0
2881361,user_20341,2022-10-25,6,0,0,0,0
2881362,user_20341,2022-11-01,1,0,0,0,0
2881363,user_20341,2022-11-17,2,0,0,0,0


# Gathering aggregated info from train_action_df

In [17]:
train_action_df_encoded['action_date'] = pd.to_datetime(train_action_df_encoded['action_date'])

# Sort the DataFrame by 'user_id' and 'action_date' to ensure the correct order for diff calculation
actions_sorted = train_action_df_encoded.sort_values(by=['user_id', 'action_date'])

# Calculate the difference in days between consecutive actions within each 'user_id'
actions_sorted['diff_days'] = actions_sorted.groupby('user_id')['action_date'].diff().dt.days


def min_max_dates_diff(group):
    return (group.max() - group.min()).days


grouped_actions = actions_sorted.groupby('user_id').agg(
    action_1 = pd.NamedAgg(column='action_1', aggfunc='sum'),
    action_2 = pd.NamedAgg(column='action_2', aggfunc='sum'),
    action_3 = pd.NamedAgg(column='action_3', aggfunc='sum'),
    action_4 = pd.NamedAgg(column='action_4', aggfunc='sum'),
    action_5 = pd.NamedAgg(column='action_5', aggfunc='sum'),
    mean_days_between_actions=pd.NamedAgg(column='diff_days', aggfunc='mean'),
    min_max_date_range = pd.NamedAgg(column='action_date', aggfunc=min_max_dates_diff)
).reset_index()
target_users = train_pay_df[train_pay_df['period']==213]['user_id'].unique()
grouped_actions['target'] = 0

# Update "target" column to 1 for target users
grouped_actions.loc[grouped_actions['user_id'].isin(target_users), 'target'] = 1
grouped_actions

,user_id,action_1,action_2,action_3,action_4,action_5,mean_days_between_actions,min_max_date_range,target
0,user_1,0,1,1,0,0,3.000000,3,0
1,user_10,0,1,1,0,0,0.000000,0,0
2,user_1000,0,4,3,1,0,38.857143,272,0
3,user_10000,0,5,4,0,0,0.500000,1,0
4,user_10001,10,668,129,9,0,1.892670,723,0
...,...,...,...,...,...,...,...,...,...
42868,user_9994,0,68,13,10,3,5.032787,307,0
42869,user_9996,0,42,7,0,0,11.540541,427,0
42870,user_9997,0,35,2,1,0,20.451613,634,0
42871,user_9998,0,5,3,0,0,4.333333,13,0


# Getting aggregated info from train_pay_df

In [18]:
# IN THE FUTURE ADD MEAN OF "How far in advance do users pay the tariff?"
grouped_tarrifs = train_pay_df_encoded.groupby('user_id').apply(lambda x: x.iloc[:, 1:-1].sum()).reset_index()
grouped_earliness = train_pay_df_encoded[['user_id','earliness_of_pay_date']].groupby('user_id').mean().reset_index()['earliness_of_pay_date']
grouped_pay_df = pd.concat([grouped_tarrifs,grouped_earliness],axis=1)
grouped_pay_df

,user_id,tariff_1,tariff_10,tariff_11,tariff_12,tariff_13,tariff_14,tariff_15,tariff_16,tariff_17,...,tariff_21,tariff_24,tariff_25,tariff_26,tariff_3,tariff_4,tariff_6,tariff_7,tariff_9,earliness_of_pay_date
0,user_1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-13.000000
1,user_10,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,-69.000000
2,user_1000,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,-26.500000
3,user_10000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-40.000000
4,user_10001,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,4,0,88.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43035,user_9994,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,-54.250000
43036,user_9996,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-30.000000
43037,user_9997,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,-38.000000
43038,user_9998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,-67.000000


# Getting final_df by merging aggregated stats from "train_action_df" and "train_pay_df"

In [81]:
final_df = pd.merge(grouped_pay_df, grouped_actions, on='user_id', how='left').fillna(0)
final_df

,user_id,tariff_1,tariff_10,tariff_11,tariff_12,tariff_13,tariff_14,tariff_15,tariff_16,tariff_17,...,tariff_9,earliness_of_pay_date,action_1,action_2,action_3,action_4,action_5,mean_days_between_actions,min_max_date_range,target
0,user_1,0,0,0,0,0,0,1,0,0,...,0,-13.000000,0.0,1.0,1.0,0.0,0.0,3.000000,3.0,0.0
1,user_10,0,0,0,0,0,0,0,0,0,...,0,-69.000000,0.0,1.0,1.0,0.0,0.0,0.000000,0.0,0.0
2,user_1000,0,1,0,0,0,0,0,0,0,...,0,-26.500000,0.0,4.0,3.0,1.0,0.0,38.857143,272.0,0.0
3,user_10000,0,0,0,0,0,0,1,0,0,...,0,-40.000000,0.0,5.0,4.0,0.0,0.0,0.500000,1.0,0.0
4,user_10001,0,0,0,0,0,0,2,0,0,...,0,88.833333,10.0,668.0,129.0,9.0,0.0,1.892670,723.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43035,user_9994,0,0,0,0,0,0,0,0,0,...,0,-54.250000,0.0,68.0,13.0,10.0,3.0,5.032787,307.0,0.0
43036,user_9996,0,0,0,0,0,0,1,0,0,...,0,-30.000000,0.0,42.0,7.0,0.0,0.0,11.540541,427.0,0.0
43037,user_9997,0,0,0,0,0,0,0,0,0,...,0,-38.000000,0.0,35.0,2.0,1.0,0.0,20.451613,634.0,0.0
43038,user_9998,0,0,0,0,0,0,0,0,0,...,0,-67.000000,0.0,5.0,3.0,0.0,0.0,4.333333,13.0,0.0


# Train_test_split and modeling

In [82]:
#!pip install xgboost

In [97]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score



test_users = pd.read_csv('test_df.csv')['user_id']
test_df = final_df[final_df['user_id'].isin(test_users)].drop(['user_id'],axis=1)
train_df = final_df[~final_df['user_id'].isin(test_users)].drop(['user_id'],axis=1)

X_test = test_df.drop(['target'],axis=1)
y_test = test_df['target']
X_train = train_df.drop(['target'],axis=1)
y_train = train_df['target']

# Convert the data into DMatrix format, which is optimized for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define hyperparameters for XGBoost
params = {
    'objective': 'binary:logistic',  # Objective for binary classification
    'max_depth': 1,  # Maximum depth of each tree
    'eta': 0.3,  # Learning rate
    'subsample': 0.7,  # Subsample ratio of the training instances
    'colsample_bytree': 0.7,  # Subsample ratio of features
    'eval_metric': 'error'  # Evaluation metric (binary classification error rate)
}

# Train the XGBoost model
num_round = 100  # Number of boosting rounds
model = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
pred_probs = model.predict(dtest)

# Calculate accuracy on the test set
true_test_labels = dtest.get_label()

roc_auc = roc_auc_score(true_test_labels, pred_probs)
print("roc_auc_score:", roc_auc)
pd.DataFrame({'user_id':test_users,'proba':pred_probs}).to_csv('predicted_df.csv',index=False)

roc_auc_score: 0.8350230709316606
